# #320 Bowstring Arch
<i>Steel bowstring arch bridge</i>
***

In [ ]:
'''Inputs - Units are kN,m'''
mesh_size = 5

transverse_spacing = 16     # Spacing of the two main arches
arch_span = 187             # Span of the two main arches
arch_rise = 22              # Vertical rise
arch_lateral_offset = 3     # Positive inwards

cantilever_outstand = 2     # Deck cantilever
include_transverse_bracing = True   # Transverse bracing between the arches
include_cross_bracing = True   # Transverse cross bracing between the arches
tapered_arch_section = True # Main arches are tapered

no_hangers = 20             # Hangers are vertical, but could be Nielsen (V-Shaped/Inclined) or Network (criss-crossed)


do_solve = False          # Carry out the analysis once the model is constructed

# Loads
surf_load_intensity = -2.5
settlement = -0.01 # m
shrinkage_strain = -0.369764E-3 
construction_load = 10.0 # Construction load (N/mm) used in staged buckling analysis applied to the top flanges


# Temperatures
temperature_increase = 20
temperature_decrease = -20



#### Connect to LUSAS Modeller and create a new model

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
import pandas as pd
import numpy as np
from LPI import *
lusas = get_lusas_modeller()
# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

# Initialise the helpers module with a reference to LUSAS
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# Create a new model
lusas.newProject("Structural", "Bow String Bridge.mdl")
# Get a reference to the current model database for convenience
db = lusas.database() 
# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Units
db.setModelUnits("kN,m,t,s,C")

Determine the geometry

In [ ]:
def get_z(x:float)->float:
    x = x - arch_span/2
    return arch_rise * (1 - (4*x*x) / (arch_span*arch_span))

def get_y(x:float)->float:
    x = x - arch_span/2
    return arch_lateral_offset * (1 - (4*x*x) / (arch_span*arch_span))

In [ ]:
x_coords = [0] + np.add.accumulate([arch_span / no_hangers] * no_hangers).tolist()
z_coords = [get_z(x) for x in x_coords] 

Create the model geometry

In [ ]:
# When calling the LPI from an external process as we are here a speedup can be gained in v22 and later by disabling the UI
if lusas.getMajorVersionNumber() >=22:
    lusas.enableUI(False)
    db.beginCommandBatch("Wizard")

In [ ]:
deck_points_1 = [Helpers.create_point(x, -transverse_spacing/2, 0) for x in x_coords]
deck_points_2 = [Helpers.create_point(x,  transverse_spacing/2, 0) for x in x_coords]

outstand_points_1 = [Helpers.create_point(x, -transverse_spacing/2-cantilever_outstand, 0) for x in x_coords]
outstand_points_2 = [Helpers.create_point(x,  transverse_spacing/2+cantilever_outstand, 0) for x in x_coords]

arch_points_1 = [Helpers.create_point(x, -transverse_spacing/2 + get_y(x), get_z(x)) for x in x_coords]
arch_points_2 = [Helpers.create_point(x,  transverse_spacing/2 - get_y(x), get_z(x)) for x in x_coords]

In [ ]:
deck_lines_1 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(deck_points_1, deck_points_1[1:])]
deck_lines_2 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(deck_points_2, deck_points_2[1:])]
deck_lines_t = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(deck_points_1, deck_points_2)]

outstand_lines_1 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(deck_points_1, outstand_points_1)]
outstand_lines_2 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(deck_points_2, outstand_points_2)]

hangers_lines_1 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(arch_points_1[1:-1], deck_points_1[1:-1])]
hangers_lines_2 = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(arch_points_2[1:-1], deck_points_2[1:-1])]

if(include_transverse_bracing):
    brace_lines = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(arch_points_1[1:-1], arch_points_2[1:-1])]
    if(include_cross_bracing):
        brace_lines.extend( [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(arch_points_1[1:-1], arch_points_2[2:-1])] )
        brace_lines.extend( [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(arch_points_2[1:-1], arch_points_1[2:-1])] )

Create the arch lines as splines

In [ ]:
arch_lines1 = Helpers.create_spline_from_points(arch_points_1, splitLine=True)
arch_lines2 = Helpers.create_spline_from_points(arch_points_2, splitLine=True)

Create model attributes

In [ ]:
# Stiffener mesh attribute
curved_beam_mesh_attr = db.createMeshLine("Curved Beams").setSize("BMI31", mesh_size)
straight_beam_mesh_attr = db.createMeshLine("Straight Beams").setSize("BMI21", mesh_size)

# Bar mesh attributes
bar_mesh_attr = db.createMeshLine("Bar Mesh").setNumber("BRS2", 1)

# Gusset/stiffener mesh attribute
shell_mesh_attr = db.createMeshSurface(f"Shell Mesh {mesh_size}mm").setRegularSize("QTS4", mesh_size, True)

# Steel Material attribute
steel_material = db.createIsotropicMaterial("Steel", 200e6, 0.3, 7.8, 10e-6)

# Arch geometic attribute
if(tapered_arch_section):
    util1 = db.createParametricSection("Arch section 1").setType("Rectangular Hollow")
    util1.setDimensions(['B', 'D', 't', 'ri', 'ro'], [2.0, 3.0, 0.06, 0.05, 0.05])
    util2 = db.createParametricSection("Arch section 2").setType("Rectangular Hollow")
    util2.setDimensions(['B', 'D', 't', 'ri', 'ro'], [2.0, 2.0, 0.06, 0.05, 0.05])

    arch_section_attr = db.createGeometricLine("Arch section")
    arch_section_attr.setValue("elementType", "3D Thick Beam")
    arch_section_attr.setMultipleVarying(True)
    arch_section_attr.setNumberOfSections(2)
    arch_section_attr.setValue("interpMethod", "Use Section Calculator")
    arch_section_attr.setFromLibrary("Utilities", "",  util1.getName(), 0, 0, 0)
    arch_section_attr.setFromLibrary("Utilities", "",  util2.getName(), 0, 0, 1)
    arch_section_attr.setSpecifyInterp(True)
    arch_section_attr.setEqualSpacing(False)
    arch_section_attr.setSymmetry(True)
    arch_section_attr.setDistanceType("Parametric")
    arch_section_attr.setSectionName(util1.getName(), 0)
    arch_section_attr.setInterpolation("Constant", 0.0, 0)
    arch_section_attr.setSectionName(util2.getName(), 1)
    arch_section_attr.setInterpolation("Cubic", 0.5, 1)
    arch_section_attr.setVerticalAlignment("CenterToCenter")
    arch_section_attr.setHorizontalAlignment("CenterToCenter")
    arch_section_attr.setAlignmentSection(0)
else:
    arch_section_attr = Helpers.create_hollow_rectangular_section(db, "Arch section", 0.5, 1.0, 0.25, 0.05, 0.05)

# Transverse brace section
if (include_transverse_bracing):
    arch_transverse_section_attr = Helpers.create_hollow_rectangular_section(db, "Arch transverse brace", 1.0, 1.0, 0.04, 0.0, 0.0)

# Longiutudinal tie beam
deck_long_section_attr = Helpers.create_equal_I_section(db, "Deck tie section", 1.2, 2.5, 0.06, 0.04)
deck_long_section_attr.setEccentricityOrigin("Centroid", "Fibre", "", "I1")

# Transverse deck sections
deck_trans_section_attr = db.createGeometricLine("Deck trans section")
deck_trans_section_attr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "914x419x388kg UB", 0, 0, 0)
deck_trans_section_attr.setEccentricityOrigin("Centroid", "Fibre", "", "I1")

# Cantilever section
cantilever_section_attr = db.createGeometricLine("Cantilever section")
cantilever_section_attr.setNumberOfSections(2)
cantilever_section_attr.setValue("interpMethod", "Use Section Calculator")
cantilever_section_attr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "914x419x388kg UB", 0, 0, 0)
cantilever_section_attr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "533x210x82kg UB", 0, 0, 1)
cantilever_section_attr.setVerticalAlignment("TopToTop")
cantilever_section_attr.setHorizontalAlignment("CenterToCenter")
cantilever_section_attr.setAlignmentSection(0)
cantilever_section_attr.setEccentricityOrigin("Centroid", "Fibre", "", "I1")

# Cable section
hanger_section_attr = Helpers.create_circular_section(db, "Hanger", 0.1)

# Support attributes
pinned = db.createSupportStructural("Fixed in XYZ").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F")
slide  = db.createSupportStructural("Fixed in YZ" ).setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")

In [ ]:
# Mesh assignments
curved_beam_mesh_attr.assignTo(arch_lines1)
curved_beam_mesh_attr.assignTo(arch_lines2)

if(include_transverse_bracing):
    straight_beam_mesh_attr.assignTo(brace_lines)

straight_beam_mesh_attr.assignTo(deck_lines_1)
straight_beam_mesh_attr.assignTo(deck_lines_2)
straight_beam_mesh_attr.assignTo(deck_lines_t)
straight_beam_mesh_attr.assignTo(outstand_lines_1)
straight_beam_mesh_attr.assignTo(outstand_lines_2)

bar_mesh_attr.assignTo(hangers_lines_1)
bar_mesh_attr.assignTo(hangers_lines_2)

In [ ]:
# Section assignments
assign = lusas.assignment().setAllDefaults()
assign.setSelectionNone()
assign.addToSelection("Line")
assign.setMultipleId(1)
arch_section_attr.assignTo(arch_lines1, assign)
assign.setMultipleId(2)
arch_section_attr.assignTo(arch_lines2, assign)

if(include_transverse_bracing):
    arch_transverse_section_attr.assignTo(brace_lines)

deck_long_section_attr.assignTo(deck_lines_1)
deck_long_section_attr.assignTo(deck_lines_2)

deck_trans_section_attr.assignTo(deck_lines_t)
cantilever_section_attr.assignTo(outstand_lines_1)
cantilever_section_attr.assignTo(outstand_lines_2)

hanger_section_attr.assignTo(hangers_lines_1)
hanger_section_attr.assignTo(hangers_lines_2)

In [ ]:
# Material assignments
steel_material.assignTo("Lines")

In [ ]:
# Support assignments
pinned.assignTo(deck_points_1[0])
pinned.assignTo(deck_points_2[0])

slide.assignTo(deck_points_1[-1])
slide.assignTo(deck_points_2[-1])

In [ ]:
# Reference path
y = 0.0
ref_path = db.createReferencePath("Lane 1")
ref_path.getDefn().addStraightV(0.0, y, 0.0, arch_span, y, 0.0)
ref_path.getDefn().setFacetData(20, 6, -1.0, 3.0, -1.0)
ref_path.getDefn().setSmoothing(False)
ref_path.getDefn().setFilletType(0, True)
ref_path.setDistanceStartOfPath(0.0)
ref_path.setTransverseDirection("Perpendicular")
ref_path.updateDefinition()


In [ ]:
# Create a search area attribute for just the nodes in the deck slab
search_area_attr = db.createSearchArea("Deck")
# Create a set of objects with the deck surface geometric attribute assigned to them
surfaces = lusas.newObjectSet().add(deck_lines_1).add(deck_lines_2).add(deck_lines_t)
# Assign the search area attribute to the same surfaces
search_area_attr.assignTo(surfaces, lusas.assignment().setAllDefaults())


# Loading

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# The loadcase will be used only to test the model is working correctly
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)
db.getAnalysis("Analysis 1").setName("00 Base")

## Permanent loads

In [ ]:
# Create a loadcase for the deck concrete loads to be applied to the beams in Phase 1
deck_conc_loadcase = db.createLoadcase("Deck Concrete")
# Set the loadcase in the default assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(deck_conc_loadcase)

# TODO


In [ ]:
# Create a loadcase for the surfacing loads to be applied to the entire deck surface in Phase 2
surfacing_loadcase = db.createLoadcase("Surfacing")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(surfacing_loadcase)

# TODO


## Environmental loads

In [ ]:
# Create a loadcases for temperature
temp_warm_loadcase = db.createLoadcase("Temperature Warming")
temp_cool_loadcase = db.createLoadcase("Temperature Cooling")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Lines", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Lines", assignment)

In [ ]:
# Wind
wind_horiz_loadcase = db.createLoadcase("Wind Y")

if tapered_arch_section:
    wind_load_attr = db.createLoadingBeamDistributed("Wind load")
    wind_load_attr.setBeamDistributed("parametric", "global", "beam")
    wind_load_attr.addRow(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)
else:
    # Prjected load is not available on tapered beams
    wind_load_attr = db.createLoadingBeamProjectedPressure("Wind load")
    wind_load_attr.setLoadDirection("globalY").setLoading(200).setLoadingWidth("factor", 1.0)

# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(wind_horiz_loadcase)
wind_load_attr.assignTo(arch_lines1, assignment)
wind_load_attr.assignTo(deck_lines_1, assignment)

# Vehicle loading

In [ ]:
# #Create a discrete load attribute which defines a group of point loads
# vehicle_attr = db.createLoadingDiscretePoint("HSLM-A9")
# # Specify the load direction (Z), the projection vector (Z)
# vehicle_attr.setDiscrete("Z", [0,0,1])
# # Add point loads for each wheel of the vehicle
# vehicle_attr.addRow(0, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(0, 0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-3, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-3, 0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-14, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-14, 0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-17, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-17, 0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-20.525, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-20.525, 0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-22.525, -0.7175, 0.0, -105.0)
# vehicle_attr.addRow(-22.525, 0.7175, 0.0, -105.0)


In [ ]:
# # Create a moving load analysis which uses the T44 Vehicle defined above
# analysis = db.createAnalysisStaticMovingLoad("Phase 3 - Moving load lane 1")
# # The moving load with follow the reference path named "Lane 1"
# analysis.setStaticMovingLoadAnalysis("HSLM-A9", "Lane 1", "Deck", "area", "All", "3D", "horizontal", "Exclude All Load", "Incremental distance", 0, 1.0, "Foward", False)
# analysis.setDefinitionMenuID(837, None ,True)


In [ ]:
# Mesh the model before makiing influence assignments
db.resetMesh()
db.updateMesh()

In [ ]:
# Influence analysis
inf_analysis = db.createAnalysisDirectMethodInf("DMI")
inf_analysis.setLoadDirection("Vertical")
inf_analysis.setLoadMagnitude(1.0E3)
inf_analysis.setSearchArea("Deck")
inf_analysis.setSearchAssignType("area")
inf_analysis.setIsGrillageType(0)
inf_analysis.setGridCentreline("")
inf_analysis.setIncludeMidSideNodes(False)
inf_analysis.setSelectedResultsGroup("assignments")
inf_analysis.setSelectedElementOutputGroup("all")
inf_analysis.setSelectedNodeOutputGroup("all")

In [ ]:
# Influence attributes for beams
influ_env_attr = db.createInfluenceEnvelope("Beams")
influ_env_attr.setResultsTransformGlobal()
influ_env_attr.setEntity("Force/Moment - Thick 3D Beam")
influ_env_attr.addComponentNames(["Fx", "Fy", "Fz", "Mx", "My", "Mz"])
influ_env_attr.includeCoincident(True)

lusas.assignment().setAllDefaults().setLoadsetOff()
influ_env_attr.assignTo(deck_lines_1)
influ_env_attr.assignTo(deck_lines_2)
influ_env_attr.assignTo(arch_lines1)
influ_env_attr.assignTo(arch_lines2)

# Cable Tuning Analysis

In [ ]:
# Create a cable tuning analysis
cable_tuning_analysis = db.createAnalysisCableTuning("Cable tuning")
# With an exact solution
cable_tuning_analysis.setOptimisation("Exact")

# Consider all cables
for side in [hangers_lines_1, hangers_lines_2]:
    for line in side:
        cable_tuning_analysis.addCable(line.getName(), 1.0)

# For the self weight loadcase
cable_tuning_analysis.addEntry("constant", "positive only", 1.0, 1.0, 1)

# In which we want to find a zero displacement for all points in the deck at the cables.
# Note for an exact solution the number of constraints must macth the number of cables
for points in [deck_points_1, deck_points_2]:
    for p in points[1:-1]:
        cable_tuning_analysis.addTarget("Point", p.getID(), "Displacement", "DZ", "=", 0.0)

## Solve

Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [ ]:
# Solve all the analyses
if do_solve:
    for analysis in db.getAnalyses():
        analysis.solve(True)
    db.openAllResults(False)

In [ ]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", False)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()

In [ ]:
# Re-enable the UI. Note if the script failed prior to this, this cell must be run manually to re-enable the UI
lusas.enableUI(True)
db.closeCommandBatch()